# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=True, enhanced=False)


Loading dataset:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]


# Methods

# Main

In [5]:
dataframe = data_loader.processed_incidents_df.copy()

dataframe = data_preprocessor._remove_incomplete_years(dataframe)
dataframe = data_preprocessor._remove_outside_region(dataframe)

dataframe.info()


<class 'pandas.core.frame.DataFrame'>
Index: 442004 entries, 5199 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  441996 non-null  object        
 1   resource_id                    442004 non-null  object        
 2   resource_type                  442004 non-null  object        
 3   time_call_received             442004 non-null  datetime64[ns]
 4   time_call_processed            442004 non-null  datetime64[ns]
 5   time_ambulance_notified        441782 non-null  datetime64[ns]
 6   time_dispatch                  428910 non-null  datetime64[ns]
 7   time_arrival_scene             390275 non-null  datetime64[ns]
 8   time_departure_scene           325959 non-null  datetime64[ns]
 9   time_arrival_hospital          325374 non-null  datetime64[ns]
 10  time_available                 441981 non-null  datetime64[ns]
 11  gr

In [6]:
print(dataframe.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe.duplicated(subset=['time_call_received', 'grid_id']).sum())


82935
83135


In [7]:
print(dataframe.duplicated(subset=["resource_id", 'time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe.duplicated(subset=["resource_id", 'time_call_received', 'grid_id']).sum())


25316
25336


In [8]:
dataframe = data_preprocessor._remove_duplicates(dataframe)

dataframe.info()


<class 'pandas.core.frame.DataFrame'>
Index: 421678 entries, 5199 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  421670 non-null  object        
 1   resource_id                    421678 non-null  object        
 2   resource_type                  421678 non-null  object        
 3   time_call_received             421678 non-null  datetime64[ns]
 4   time_call_processed            421678 non-null  datetime64[ns]
 5   time_ambulance_notified        421472 non-null  datetime64[ns]
 6   time_dispatch                  409579 non-null  datetime64[ns]
 7   time_arrival_scene             373982 non-null  datetime64[ns]
 8   time_departure_scene           313286 non-null  datetime64[ns]
 9   time_arrival_hospital          312748 non-null  datetime64[ns]
 10  time_available                 421655 non-null  datetime64[ns]
 11  gr

In [9]:
print(dataframe.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe.duplicated(subset=['time_call_received', 'grid_id']).sum())


62609
62809


In [10]:
print(dataframe.duplicated(subset=["resource_id", 'time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe.duplicated(subset=["resource_id", 'time_call_received', 'grid_id']).sum())


4990
5010


In [11]:
dataframe["resource_type"].value_counts()

resource_type
Ambulanse         404268
Operativ Leder     12612
Legebil             4617
Syketransport        133
Akuttbil              48
Name: count, dtype: int64

In [12]:
dataframe_test = dataframe[(dataframe["triage_impression_during_call"] == "A") | (dataframe["triage_impression_during_call"] == "H")]

print(dataframe_test.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe_test.duplicated(subset=['time_call_received', 'grid_id']).sum())

dataframe_test["resource_type"].value_counts()


52981
52989


resource_type
Ambulanse         320020
Operativ Leder     12440
Legebil             4507
Akuttbil              46
Syketransport          3
Name: count, dtype: int64

In [13]:
dataframe_test = dataframe.dropna(subset=["time_ambulance_notified", "time_dispatch", "time_arrival_scene", "time_available"])

print(dataframe_test.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe_test.duplicated(subset=['time_call_received', 'grid_id']).sum())

print(dataframe_test["resource_type"].value_counts())


28640
28825
resource_type
Ambulanse         367164
Legebil             2873
Operativ Leder      2787
Syketransport        123
Akuttbil              34
Name: count, dtype: int64


In [14]:
dataframe_test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 372981 entries, 89162 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  372975 non-null  object        
 1   resource_id                    372981 non-null  object        
 2   resource_type                  372981 non-null  object        
 3   time_call_received             372981 non-null  datetime64[ns]
 4   time_call_processed            372981 non-null  datetime64[ns]
 5   time_ambulance_notified        372981 non-null  datetime64[ns]
 6   time_dispatch                  372981 non-null  datetime64[ns]
 7   time_arrival_scene             372981 non-null  datetime64[ns]
 8   time_departure_scene           305201 non-null  datetime64[ns]
 9   time_arrival_hospital          304439 non-null  datetime64[ns]
 10  time_available                 372981 non-null  datetime64[ns]
 11  g

In [15]:
dataframe_test_2 = dataframe_test[dataframe_test["resource_type"] == "Ambulanse"]

print(dataframe_test_2.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate']).sum())
print(dataframe_test_2.duplicated(subset=['time_call_received', 'grid_id']).sum())

print(dataframe_test_2["resource_type"].value_counts())

dataframe_test.info()


23233
23418
resource_type
Ambulanse    367164
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 372981 entries, 89162 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  372975 non-null  object        
 1   resource_id                    372981 non-null  object        
 2   resource_type                  372981 non-null  object        
 3   time_call_received             372981 non-null  datetime64[ns]
 4   time_call_processed            372981 non-null  datetime64[ns]
 5   time_ambulance_notified        372981 non-null  datetime64[ns]
 6   time_dispatch                  372981 non-null  datetime64[ns]
 7   time_arrival_scene             372981 non-null  datetime64[ns]
 8   time_departure_scene           305201 non-null  datetime64[ns]
 9   time_arrival_hospital          304439 non-null  datetime64[ns]
 1

In [16]:
datetime_columns = [
    "time_call_received", "time_call_processed", "time_ambulance_notified",
    "time_dispatch", "time_arrival_scene", "time_departure_scene",
    "time_arrival_hospital", "time_available"
]

# Count rows that violate the sequence
violations = 0
for i in range(len(datetime_columns) - 1):
    first_col = datetime_columns[i]
    second_col = datetime_columns[i + 1]
    
    # Count where the first date is after the second date
    violations += dataframe_test[dataframe_test[first_col] > dataframe_test[second_col]].shape[0]

violations


35923

In [17]:
datetime_columns = [
    "time_call_received", "time_call_processed", "time_ambulance_notified",
    "time_dispatch", "time_arrival_scene", "time_departure_scene",
    "time_arrival_hospital", "time_available"
]

violations_mask = pd.Series(False, index=dataframe_test.index)

# Update the mask for each pair of datetime columns
for i in range(len(datetime_columns) - 1):
    first_col = datetime_columns[i]
    second_col = datetime_columns[i + 1]
    
    # Update the mask where the first date is after the second date
    violations_mask |= (dataframe_test[first_col] > dataframe_test[second_col])

# Count total rows with any violation
total_violations = violations_mask.sum()

total_violations


35915

In [18]:
datetime_columns = [
    "time_call_received", "time_call_processed", "time_ambulance_notified",
    "time_dispatch", "time_arrival_scene", "time_departure_scene",
    "time_arrival_hospital", "time_available"
]

# Dictionary to store the count of violations for each column pair
violations_count = {}

# Count violations for each pair of datetime columns
for i in range(len(datetime_columns) - 1):
    first_col = datetime_columns[i]
    second_col = datetime_columns[i + 1]
    
    # Count where the first date is after the second date
    count = dataframe_test[dataframe_test[first_col] > dataframe_test[second_col]].shape[0]
    violations_count[f"{first_col} to {second_col}"] = count

# Sorting the dictionary by the count of violations
sorted_violations = sorted(violations_count.items(), key=lambda x: x[1], reverse=True)

sorted_violations

[('time_call_received to time_call_processed', 31978),
 ('time_call_processed to time_ambulance_notified', 3910),
 ('time_dispatch to time_arrival_scene', 21),
 ('time_arrival_hospital to time_available', 10),
 ('time_ambulance_notified to time_dispatch', 4),
 ('time_arrival_scene to time_departure_scene', 0),
 ('time_departure_scene to time_arrival_hospital', 0)]